In [80]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import scipy.sparse as sp
import torch
import torch.optim as optim

In [81]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [82]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)

    return labels_onehot


def load_data(path="../data/cora", dataset="cora"):
    print(f"Loading {dataset} dataset...")

    idx_features_labels = np.genfromtxt(
        f"{path}/{dataset}.content", dtype=np.dtype(str)
    )
    #     array([['31336', '0', '0', ..., '0', '0', 'Neural_Networks'],
    #        ['1061127', '0', '0', ..., '0', '0', 'Rule_Learning'],
    #        ['1106406', '0', '0', ..., '0', '0', 'Reinforcement_Learning'],
    #        ...,
    #        ['1128978', '0', '0', ..., '0', '0', 'Genetic_Algorithms'],
    #        ['117328', '0', '0', ..., '0', '0', 'Case_Based'],
    #        ['24043', '0', '0', ..., '0', '0', 'Neural_Networks']],dtype='<U22')
    # idx_features_labels는 1:-1이 Features / -1이 Labels로 나타난다.
    # Graph를 구축하기 위한 관계도는 .cites에 위치

    # 각 Features를 csr_matrix(compressed Sparse Row Matrix)로 변환
    # 각 Labels를 One Hot Encoding으로 변환
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # csr_matrix:
    # coo_matrix: A sparse matrix in COOrdinate format

    # Build Graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)  # index for papers
    idx_map = {j: i for i, j in enumerate(idx)}                # index의 순차적 index

    edges_unordered = np.genfromtxt(
        f"{path}/{dataset}.cites", dtype=np.int32
    )                                                          # edges_unordered: 각 Paper가 가리키고 있는 Paper
    edges = np.array(
        list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32
    ).reshape(edges_unordered.shape)                           # edges들을 고유 index가 아닌 개수에 맞는 index 변환
    
    
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)
    
    # Build Symmetric Adjacency Matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))
    # Normalize의 과정 속에 Degree의 -1/2승을 곱해주는 것이 있는데, 이를 위해
    # 먼저 Adjacency Matrix A에 eye matrix를 더해주는 Renormalization Trick을 적용하는 것을 볼 수 있습니다.
    
    idx_train = range(140)            # Train은 0~140
    idx_val = range(200, 500)         # Validation은 200~500
    idx_test = range(500, 1500)       # Test 는 500~1500 Index를 가진 데이터

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

In [83]:
import math

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module


class GraphConvolution(Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [84]:
import torch.nn as nn
import torch.nn.functional as F


class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [97]:
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [ ]:
import time
from tqdm import tqdm

In [95]:
hidden = 16
dropout = 0.5
lr = 0.01
weight_decay = 5e-4
cuda = True

if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()
    
model = GCN(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout)

optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

def train(epoch):
    t = time.time()
    for idx in tqdm(range(epoch)):
        model.train()
        optimizer.zero_grad()
        output = model(features, adj)
        loss_train = F.nll_loss(output[idx_train], labels[idx_train])
        acc_train = accuracy(output[idx_train], labels[idx_train])
        loss_train.backward()
        optimizer.step()
        
        if idx % 100 == 0:
            model.eval()
            output = model(features, adj)
            loss_val = F.nll_loss(output[idx_val], labels[idx_val])
            acc_val = accuracy(output[idx_val], labels[idx_val])
            print('Epoch: {:04d}'.format(epoch+1),
                  'loss_train: {:.4f}'.format(loss_train.item()),
                  'acc_train: {:.4f}'.format(acc_train.item()),
                  'loss_val: {:.4f}'.format(loss_val.item()),
                  'acc_val: {:.4f}'.format(acc_val.item()),
                  'time: {:.4f}s'.format(time.time() - t))
            
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [98]:
train(1000)

  2%|▏         | 17/1000 [00:00<00:06, 161.78it/s]

Epoch: 1001 loss_train: 1.9435 acc_train: 0.1643 loss_val: 1.9457 acc_val: 0.1267 time: 0.0099s


 13%|█▎        | 134/1000 [00:00<00:04, 189.02it/s]

Epoch: 1001 loss_train: 0.6612 acc_train: 0.8786 loss_val: 0.8939 acc_val: 0.7967 time: 0.5628s


 23%|██▎       | 232/1000 [00:01<00:04, 188.73it/s]

...

 82%|████████▎ | 825/1000 [00:04<00:00, 200.97it/s]

Epoch: 1001 loss_train: 0.2102 acc_train: 0.9786 loss_val: 0.6208 acc_val: 0.8233 time: 4.1465s


 92%|█████████▎| 925/1000 [00:04<00:00, 180.09it/s]

Epoch: 1001 loss_train: 0.2138 acc_train: 0.9714 loss_val: 0.6206 acc_val: 0.8100 time: 4.6878s


100%|██████████| 1000/1000 [00:05<00:00, 190.58it/s]


In [99]:
test()

Test set results: loss= 0.6021 accuracy= 0.8330


In [60]:
path="../data/cora"
dataset="cora"

In [61]:
idx_features_labels = np.genfromtxt(f'{path}/{dataset}.content', dtype=np.dtype(str))

In [62]:
idx = np.array(idx_features_labels[:, 0], dtype=np.int32)

In [63]:
idx_map = {j:i for i, j in enumerate(idx)} 

In [64]:
idx_map.get(35)

163

In [65]:
edges_unordered = np.genfromtxt(f'{path}/{dataset}.cites', dtype=np.int32)

In [66]:
edges_unordered.flatten()

array([     35,    1033,      35, ...,  853118,  954315, 1155073],
      dtype=int32)

In [67]:
edges_unordered

array([[     35,    1033],
       [     35,  103482],
       [     35,  103515],
       ...,
       [ 853118, 1140289],
       [ 853155,  853118],
       [ 954315, 1155073]], dtype=int32)

In [70]:
edges = np.array(
        list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32
    ).reshape(edges_unordered.shape)
    

# Build Symmetric Adjacency Matrix
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

In [77]:
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                    shape=(labels.shape[0], labels.shape[0]),
                    dtype=np.float32)


In [78]:
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

In [79]:
adj

<2708x2708 sparse matrix of type '<class 'numpy.float32'>'
	with 10556 stored elements in Compressed Sparse Row format>

In [71]:
edges

array([[ 163,  402],
       [ 163,  659],
       [ 163, 1696],
       ...,
       [1887, 2258],
       [1902, 1887],
       [ 837, 1686]], dtype=int32)

In [68]:
labels = encode_onehot(idx_features_labels[:, -1])

In [69]:
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

NameError: name 'edges' is not defined

In [36]:
adj

<2708x2708 sparse matrix of type '<class 'numpy.float32'>'
	with 5429 stored elements in COOrdinate format>

In [30]:
idx_map.get(103482)

659

In [32]:
edges = np.array(
        list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32
    ).reshape(edges_unordered.shape)

In [ ]:
class GraphConvolution

In [ ]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()
        
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        
        self.dropout = dropout
    
    # adj: Adjacency Matrix
    # x: Feature Matrix
    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        